# PRODUCTS database

In [52]:
import pandas as pd
import re
df = pd.read_csv('PRODUCTS_orig.csv', sep='|', index_col=False)
# delete columns
df = df.drop('SKU', 1)
df = df.drop('BUY_UNIT', 1)
df = df.drop('DESCRIPTION_STATUS', 1)
df = df.drop('ORGANIC_ITEM', 1)
df = df.drop('KIRLAND_ITEM', 1)
df = df.drop('FINELINE_NUMBER', 1)
df = df.drop('ITEM_IMG', 1)
# sort
df.sort_values("EAN", inplace=True)
# dropping ALL duplicte values 
df.drop_duplicates(subset="EAN", keep=False, inplace=True)
# clean html tags
df['ITEM_DESCRIPTION'] = df['ITEM_DESCRIPTION'].apply(lambda x: re.sub('<[^<]+?>', '', x))
# filter EAN 8 length
df.EAN = df.EAN.apply(str)
df = df[df['EAN'].apply(lambda x: len(x)>=8)]
# get size, weight, quantity
# (\d*\.?\d+)\s*(gr?|kgs?)
df.describe()

,EAN,ITEM_NAME,ITEM_DESCRIPTION,CATEGORY,SUB_CATEGORY,SUB_SUB_CATEGORY,BRAND_NAME
count,250865,250853,250865,249743,249743,249743,250765
unique,250865,144382,206011,15,62,395,2637
top,7501857071388,QUESO,QUESO 1 KG.,HARDLINES,92 GROCERY,CONSIGNADO AFECTO,TOCORNAL
freq,1,861,599,45131,18751,2437,141


In [53]:
# store CATEGORY,SUB_CATEGORY,SUB_SUB_CATEGORY and BRAND_NAME in CSVs and delete their columns
CATEGORY = pd.Series(df.CATEGORY.unique()).sort_values()
CATEGORY.to_csv("category.csv")
SUB_CATEGORY = pd.Series(df.SUB_CATEGORY.unique()).sort_values()
SUB_CATEGORY.to_csv("sub_category.csv")
SUB_SUB_CATEGORY = pd.Series(df.SUB_SUB_CATEGORY.unique()).sort_values()
SUB_SUB_CATEGORY.to_csv("sub_sub_category.csv")
BRAND_NAME = pd.Series(df.BRAND_NAME.unique()).sort_values()
BRAND_NAME.to_csv("brand.csv")
len(BRAND_NAME)
# delete unecessary columns
del df['CATEGORY']
del df['SUB_CATEGORY']
del df['SUB_SUB_CATEGORY']
del df['BRAND_NAME']
df_country = pd.read_csv('gs1_countries.csv', sep='|', index_col=False)
df_country.country_code = df_country.country_code.apply(str)
# get counrty code
#df['country_code'] = df['EAN'].str[:3]
# get csv of countries


# PRODUCTOS compra mercado libre

In [54]:
import pandas as pd
import re
df1 = pd.read_csv('productos_ml1.csv', index_col=False)
# dropping ALL duplicte values 
df1.drop_duplicates(subset="EAN", keep=False, inplace=True)
# sort
#df1.sort_values("EAN", inplace=True)
# filter EAN 8 length
df1.EAN = df1.EAN.apply(str)
df1 = df1[df1['EAN'].apply(lambda x: len(x)>=8)]
df1.describe()

,EAN,producto
count,20181,20181
unique,20181,19901
top,7506295319900,BOLSA REGALO
freq,1,25


# OPEN FOODS FACTS # https://world.openfoodfacts.org/data

In [71]:
import pandas as pd
import re
fields = ['code',
 'product_name',
 #'quantity',
 #'brands',
 #'categories_en',
 #'countries_en',
 #'image_url',
]
df2 = pd.read_csv('openfoodfacts.org.products_reduced.csv', sep='\t', index_col=False, usecols=fields)
df2.rename(columns={'code': 'EAN'}, inplace=True)
# dropping ALL duplicte values 
df2.EAN = df2.EAN.apply(str)
df2.drop_duplicates(subset="EAN", keep=False, inplace=True)
# filter EAN 8 length
df2 = df2[df2['EAN'].apply(lambda x: len(x)>=8)]
df2.describe()

/home/maugb/.virtualenvs/test/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,EAN,product_name
count,1641045,1567067
unique,1641045,1039877
top,217247029453,Aceite de oliva virgen extra
freq,1,1206


In [72]:
df_join1 = pd.merge(df, df1, on='EAN', how='outer')
df_join1 = pd.merge(df2, df_join1, on='EAN', how='outer')
df_join1.describe()

,EAN,product_name,ITEM_NAME,ITEM_DESCRIPTION,producto
count,1888942,1567067,250853,250865,20181
unique,1888942,1039877,144382,206011,19901
top,217247029453,Aceite de oliva virgen extra,QUESO,QUESO 1 KG.,BOLSA REGALO
freq,1,1206,861,599,25


In [81]:
#df_join1[df_join1.producto.isnull() & df_join1.ITEM_NAME.isnull()] # ITEM_NAME y producto son null
#df_join1[df_join1.producto.notnull() & df_join1.ITEM_NAME.notnull()].describe()
#df_join1[df_join1.producto.notnull()].describe()
df_join1[df_join1.producto.notnull()].to_csv("test.csv")
